In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk
import pathlib
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import glob
from glob import glob
import os
from os import listdir
import requests
from zipfile import ZipFile
import keras.utils as image

import plotly.express as px
import seaborn as sns
import PIL
import random

random.seed(0)
import cv2
from matplotlib.image import imread

from IPython.display import display, Image

2023-12-05 14:48:06.984336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#Got this from ChatGPT


# Define the GitHub repository URL
repo_url = 'https://github.com/StevenHPatrick/breastCancerData/archive/94de9300f0851b14430654cd36089af864e6017b.zip'

# Specify the local path to save the downloaded file
zip_file_path = '/content/breast_cancer_data.zip'

# Download the dataset from GitHub
response = requests.get(repo_url)
with open(zip_file_path, 'wb') as zip_file:
    zip_file.write(response.content)

# Extract the contents of the ZIP file
extracted_path = '/content/breastCancerData-94de9300f0851b14430654cd36089af864e6017b'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Adjust the path to include the additional subdirectory
dataset_path = os.path.join(extracted_path, 'breastCancerData-94de9300f0851b14430654cd36089af864e6017b')


breast_img = glob(os.path.join(dataset_path, 'archive/IDC_regular_ps50_idx5/**/*.png'), recursive=True)

# Display the first 3 images
for img_path in breast_img[:3]:
    display(Image(filename=img_path))


In [ ]:
non_img = []
can_img = []

for img in breast_img:
    if img[-5] == '0' :
        non_img.append(img)

    elif img[-5] == '1' :
        can_img.append(img)
breast_img[0]

In [ ]:
non_num = len(non_img)
can_num = len(can_img)

total_img_num = non_num + can_num

print('Number of Images in IDC (-): {}' .format(non_num))
print('Number of Images in IDC (+) : {}' .format(can_num))
print('Total Number of Images : {}' .format(total_img_num))

In [ ]:
plt.figure(figsize = (15, 15))

some_non = np.random.randint(0, len(non_img), 18)
some_can = np.random.randint(0, len(can_img), 18)

s = 0
for num in some_non:

        img = image.load_img((non_img[num]), target_size=(100, 100))
        img = image.img_to_array(img)

        plt.subplot(6, 6, 2*s+1)
        plt.axis('off')
        plt.title('No cancer')
        plt.imshow(img.astype('uint8'))
        s += 1
s = 1
for num in some_can:
         img = image.load_img((can_img[num]), target_size=(100, 100))
         img = image.img_to_array(img)
         plt.subplot(6, 6, 2*s)
         plt.axis('off')
         plt.title('Cancer')
         plt.imshow(img.astype('uint8'))
         s += 1

In [ ]:
non_cancer_num = len(non_img)  # No cancer
cancer_num = len(can_img)   # Cancer

total_img_num = non_cancer_num + cancer_num

print('Number of Images of no cancer: {}' .format(non_cancer_num))   # images of Non cancer
print('Number of Images of cancer : {}' .format(cancer_num))   # images of cancer
print('Total Number of Images : {}' .format(total_img_num))

Data Preprocessing

In [ ]:
# Randomly sample images from two lists, 'non_cancer_imgs' and 'cancer_imgs'
some_non_img = random.sample(non_img, len(non_img))
some_can_img = random.sample(can_img, len(can_img))

# Initialize empty arrays to store image data and labels
non_img_arr = []  # Array for non-cancer images
can_img_arr = []  # Array for cancer images

# Loop through each image in the 'some_non_img' list
for img in some_non_img:
    # Read the image in color mode
    n_img = cv2.imread(img, cv2.IMREAD_COLOR)
    # Resize the image to a fixed size (50x50 pixels) using linear interpolation
    n_img_size = cv2.resize(n_img, (50, 50), interpolation=cv2.INTER_LINEAR)
    # Append the resized image and label 0 (indicating non-cancer) to the 'non_img_arr'
    non_img_arr.append([n_img_size, 0])

    # Loop through each image in the 'some_can_img' list
for img in some_can_img:
    # Read the image in color mode
    c_img = cv2.imread(img, cv2.IMREAD_COLOR)
    # Resize the image to a fixed size (50x50 pixels) using linear interpolation
    c_img_size = cv2.resize(c_img, (50, 50), interpolation=cv2.INTER_LINEAR)
    # Append the resized image and label 1 (indicating cancer) to the 'can_img_arr'
    can_img_arr.append([c_img_size, 1])